In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from threading import Thread

class StreamingThread(Thread):
    def __init__(self, ssc):
        Thread.__init__(self)
        self.ssc = ssc
    def run(self):
        ssc.start()
        ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [3]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [4]:
spark

In [5]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType

In [6]:
from difflib import unified_diff

def make_diff(old, new):
    return '\n'.join([ l for l in unified_diff(old.split('\n'), new.split('\n')) if l.startswith('+') or l.startswith('-') ])

In [7]:
globals()['models_loaded'] = False

# ------------ Replace this with a trained ML model
def predict(df):
    if any([x in df.diff.lower() for x in ['bad', 'lol', 'joke']]):
        return 'vandal'
    else:
        return 'safe'
# ------------ Replace this with a trained ML model

predict_udf = udf(predict, StringType())

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    # Tip: making a diff will probably help a lot as a feature in any model:
    diff = make_diff(df.first().text_old, df.first().text_new)
    df_withdiff = df.withColumn("diff", lit(diff))
    df_withdiff.select('diff').show()
    
    # Utilize our predict function
    df_withpreds = df_withdiff.withColumn("pred", predict_udf(
        struct([df_withdiff[x] for x in df_withdiff.columns])
    ))
    df_withpreds.show()
    
    # Normally, you wouldn't use a UDF (User Defined Function) Python function to predict (you can)
    # But an MLlib model you've built and saved with Spark
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = '***' # Replace '***' with:    [...].load('my_logistic_regression')
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model: 
    # df_result = globals()['my_model'].transform(df)
    # df_result.show()

In [8]:
ssc = StreamingContext(sc, 10)

In [9]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [10]:
# First thing you see is the conversion from the json text to a dataframe --> Can be used for featurization
# Then diff will be computed
# Then we make the prediction
ssc_t = StreamingThread(ssc)
ssc_t.start()

========= 2020-05-03 19:05:20 =========
+--------------------+-----+------------------+--------------------+--------------------+--------------------+--------------------+
|             comment|label|         name_user|            text_new|            text_old|          title_page|            url_page|
+--------------------+-----+------------------+--------------------+--------------------+--------------------+--------------------+
|→‎Year-end charts...| safe| WereSpielChequers|{{About|the Justi...|{{About|the Justi...|          Suit & Tie|//en.wikipedia.or...|
|(GR)File renamed:...| safe|       Samuele2002|{{Use Hong Kong E...|{{Use Hong Kong E...|Yau Ma Tei Fruit ...|//en.wikipedia.or...|
|→‎Notable alumni ...| safe|Howardrandallsmith|'''''The Lost Col...|'''''The Lost Col...|The Lost Colony (...|//en.wikipedia.or...|
|→‎Year-end charts...| safe| WereSpielChequers|{{Infobox song
| ...|{{Infobox song
| ...|Started from the ...|//en.wikipedia.or...|
|                    | safe|        

In [11]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----
========= 2020-05-03 19:05:30 =========
+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+
|             comment|label|        name_user|            text_new|            text_old|          title_page|            url_page|
+--------------------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+
|Attribution: text...| safe|          Diannaa|{{Use dmy dates|d...|{{Use dmy dates|d...|Danko Jones (musi...|//en.wikipedia.or...|
|spelling (Wikiped...| safe|            Cyfal|{{Hatnote|For the...|{{Hatnote|For the...|Excommunication (...|//en.wikipedia.or...|
|replaced: Masters...| safe|Chris the speller|'''Cynthia A. Bre...|'''Cynthia A. Bre...|      Cynthia Brewer|//en.wikipedia.or...|
+--------------------+-----+-----------------+--------------------+--------------------+--------------------+-------------